In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')
        

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
# Take the correct pedestal file (usually located in /sf/alvra/data/*pgroup*/res/JF_pedestals)
# Take the correct gains.h5 file (usually located in /sf/alvra/config/jungfrau/gainMaps/JF02T09V01/)

#pede_file = "/sf/alvra/data/p17983/res/JF_pedestals/pedestal_20190723_1255.JF02T09V02.res.h5"
pede_file = "/sf/alvra/data/p17983/res/JF_pedestals/pedestal_20190724_1140.JF02T09V02.res.h5"
#gain_file = "/sf/alvra/config/jungfrau/gainMaps/JF02T09V02/gains.h5"
gain_file = "/sf/alvra/config/jungfrau/gainMaps/JF02T09V02/gains.h5"

with h5py.File(gain_file, "r") as file: 
    G = file["gains"][:]
    
with h5py.File(pede_file, "r") as file: 
    P = file["gains"][:]
    mask = file["pixel_mask"][:]

print("Dimensions of G: ", G.shape)
print("Dimensions of P: ", P.shape)
print("Dimensions of mask: ", mask.shape)

# Detector name for loading module map and applying geometry
detector_name = "JF02T09V02"

Dimensions of G:  (4, 4608, 1024)
Dimensions of P:  (4, 4608, 1024)
Dimensions of mask:  (4608, 1024)


In [4]:
#DIR = "/sf/alvra/data/p17983/raw/scan_data/Ru_foil_monoscan_001/"
# DIR = "/sf/alvra/data/p17983/raw/"

scan_name = 'RuBpy3_monoscan_001'
#scan_name = 'Ru_foil_monoscan_003'
DIR = '/sf/alvra/data/p17983/raw/scan_data/' + scan_name + '/'
DIR_json = '/sf/alvra/data/p17983/res/scan_info/'


listfile = os.listdir(DIR)
number_files = len(listfile)
print ("There are",number_files,"files in the folder", DIR)

There are 164 files in the folder /sf/alvra/data/p17983/raw/scan_data/RuBpy3_monoscan_001/


In [5]:
numberSteps = 1
roi=[200,320,4850,5250]
#user sets number of frames to combine
N_shots_to_use = 100
xes_abs_scan = []
diode_abs_scan = np.empty(0)
Energy_eV = np.empty(0)
#iZero = np.empty(0)

In [6]:
for x in range(0,numberSteps):

    # filename = "Ru_foil_monoscan_001_step0020.JF02T09V02.h5"
    # bsreadfile = "Ru_foil_monoscan_001_step0020.BSREAD.h5"


    #filename_root = "Ru_foil_monoscan_001_step0"
    filename_root = "RuBpy3_monoscan_001_step0"
    # filename_root = "Ru_foil_XES_3keV_002"

    if x<10:
        filename_root = filename_root + '00'
    elif x<100:
        filename_root = filename_root + '0'
    filename_root = filename_root + str(x)


    filename = filename_root+".JF02T09V02.h5"
    bsreadfile = filename_root+".BSREAD.h5"

    print(filename)
    print(bsreadfile)

    images, pulse_ids = load_JF_data_corrected(DIR + filename, G, P, mask, nshots=N_shots_to_use)

    #this function assumes pump on/off handeling and we just forced everything into one of them and read it out
    #(DataFluo, _, IzeroFEL, _, BS_pulse_ids, _, _, _) = \
    #    load_PumpProbe_events(DIR+bsreadfile, channel_BS_pulse_ids,nshots=N_shots_to_use*2)   
    
    (DataFluo_p, DataFluo_u, IzeroFEL_p, IzeroFEL_u, \
        BS_pulse_ids_p, BS_pulse_ids_u, Energy_p, Energy_u, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIR + bsreadfile, \
                                               channel_BS_pulse_ids, channel_energy, nshots=N_shots_to_use)
    print('DataFluo_u:')
    print(DataFluo_u)

    nframes = images.shape[0]
    print('number of frames')
    print(nframes)
    # nframes = 1

    X = [a==b for a,b in zip(pulse_ids, BS_pulse_ids)]
    if sum([int(x) for x in X]) == images.shape[0]:
        print('passed pulse id check')
    else:
        print('failed pulse id check')
    # print(images.shape)

    numstds = 2 #2
    minIzero = 0.002 #0.002
    image_threshold = 1
    hot_pixel = 15

    total = 0
    # total_off = 0

    IzeroMedian = np.median(IzeroFEL)
    IzeroSTD = np.std(IzeroFEL)

    # print(images_off.shape)
    print('number of frames')
    print(nframes)

    conditionMax = IzeroFEL < IzeroMedian+numstds*IzeroSTD
    conditionMin = IzeroFEL > IzeroMedian-numstds*IzeroSTD
    conditionLow = IzeroFEL > minIzero
    condition = np.logical_and.reduce((conditionLow, conditionMin, conditionMax)).T[0]
    # if conditionMax and conditionMin and conditionLow:

    # print(condition)

    images_good = images[condition,...]
    #         image_off = ju.apply_gain_pede(image_off, G=G, P=P, pixel_mask=mask, highgain=False)
    images_thr = images_good.copy()
    images_thr[images_good < image_threshold] = 0
    images_thr[images_good > hot_pixel] = 0

    print('number of surviving frames')
    print(images_thr.shape[0])
    print(images_thr.shape)

    total = images_thr.sum(axis=0)

    total_corr = ju.apply_geometry(total, detector_name)
    print(total_corr.shape)
    total_ = total_corr[roi[0]:roi[1],roi[2]:roi[3]]
    print(total_.shape)
    xes_abs = sum(sum(total_))
    print(xes_abs)
    xes_abs_scan.append(xes_abs)
    diode_abs_scan = np.append(diode_abs_scan, DataFluo_u)
    
    Energy_u = [x for x in Energy_u if (np.abs(x) > 0)]
    Energy_eV = np.append(Energy_eV, np.mean(Energy_u))
    #iZero = np.append(iZero, np.mean(IzeroFEL_pump_total))
    

RuBpy3_monoscan_001_step0000.JF02T09V02.h5
RuBpy3_monoscan_001_step0000.BSREAD.h5
from load_JF
<class 'numpy.ndarray'>
Will apply module map: [-1 -1 -1  0  1  2  3  4  5]


/sf/alvra/anaconda/dev/lib/python3.6/site-packages/jungfrau_utils/corrections.py:202: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  res = np.stack(partial_func_to_use(i) for i in image)


TypeError: ufunc 'right_shift' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
print(diode_abs_scan)
plt.figure()
#plt.plot(Energy_eV, xes_abs_scan)
plt.plot(Energy_eV, diode_abs_scan)